## EM算法
EM算法指的是最大期望算法(Expectation Maximization Algorithm),是一种迭代算法，用于含有隐变量的概率参数模型的最大似然或极大后验概率估计。

### EM算法的引入
#### 例(三硬币模型)
假设有三枚硬币A，B，C，每个硬币正面出现的概率分别为$\pi,p,q$。进行如下的掷硬币实验，先掷硬币A，如果正面朝上则选硬币B，反面朝上选C；然后投掷选择的硬币，出现的结果正面为1，反面为0；独立地重复n次实验，这里n=10，结果如下：
$$ 1,1,0,1,0,0,1,0,1,1$$
假设智能观测到掷硬币的结果，不能观测掷硬币的过程。问如果估计三硬币正面出现的概率，即三硬币模型的参数？
- 解答：  
针对某个输出值y，它在参数$\theta=(\pi,p,q)$下的概率分布为：  
\begin{align}
p(y|\theta)&=\sum_zp(y,z|\theta)=\sum_zp(z|\theta)p(y|z,\theta)\\
&=\pi p^y(1-p)^{1-y}+(1-\pi)q^y(1-q)^{1-y}
\end{align}
将观测数据表示为$Y=(Y_1,Y_2,\cdots,Y_n)^T$,未观测数据表示为$Z=(Z_1,Z_2,\cdots,Z_n)^T$,则，观测数据的似然函数为：  
$$P(Y|\theta)=\sum_ZP(Z|\theta)P(Y|Z,\theta)$$
或者可以写成：  
\begin{align}
P(Y|\theta)&=\prod_{i=1}^np(y_i|\theta)\\
&=\prod_{i=1}^np(z_i|\theta)p(y_i|z_i,\theta)\\
&=\prod_{i=1}^n \pi p^{y_i}(1-p)^{1-y_i}+(1-\pi)q^{y_i}(1-q)^{1-y_i}
\end{align}
直接连乘的似然函数求导太复杂，所以一般用极大似然估计都会转化成对数似然函数。  
本题的目标就变成了求解参数$\theta$的对数极大似然估计，即：  
$$\hat{\theta}=\arg\max\limits_{\theta}\log P(Y|\theta)$$  
这个问题没有解析解，只有通过迭代的方式求解。  
- 接下来的部分将介绍如何求解


### 1.Jensen不等式
#### 1.1 凹凸函数
定义：如果函数总是位于任何一条弦的下方，则该函数是凸的；如果函数总是位于任何一条弦的上方，则函数是凹的。  
用数学语言表述如下：  
如果函数f在某个区间上存在非负(正)的二阶导数，即$f''(x)\ge 0$，则f为该区间的凸函数。当x是向量时，如果其Hessian矩阵H是半正定的，那么f是凸函数。如果$f''(x)\ge 0$或者$H>0$,那么f是严格凸函数。  
  
凸函数的例子有$x^2,|x|,e^x,x \log x(x\ge 0)$;凹函数的例子有log x,$\sqrt{x}(x\ge 0)$等。
#### 1.2 Jensen不等式：
如果f是凸函数，X是随机变量，那么
$$E[f(X)]\ge f(E[X])$$
特别地，如果f是严格凸的，等式成立当且仅当P(X=E[X])=1,即X是常量。   
Jensen不等式应用于凹函数时，不等号方向反向。

### 2.EM算法
> - 一般地，用Y表示观测随机变量的数据，Z表示隐随机变量的数据，Y和Z连在一起称为完全数据，观测数据Y又称为不完全数据。不完全数据Y的似然函数是$\log P(Y|\theta)$,完全数据的对数似然函数为$\log P(Y,Z|\theta)$。  
> - EM算法通过迭代求$L(\theta)=\log P(Y|\theta)$的极大似然估计。

给定训练样本为${\{y_1,y_2,\cdots,y_n}\}$,样例之间相互独立，我们想找到每个样例的隐含类别z，能使得p(y,z)最大。(这里$z_i=j$可以看成样本$y_i$被划分成j类) 记模型参数为$\theta$。  
训练样本的对数极大似然函数如下：  
\begin{align}
L(\theta)&=\sum_{i=1}^n\log p(y_i;\theta)\\
&=\sum_{i=1}^n\log\sum_{z_i}p(y_i,z_i;\theta)
\end{align}
第一步是对极大似然函数取对数，第二步是对每个样例的每个可能的类别j求联合分布概率和，但是直接求$\theta$一般比较困难，因为有隐变量z的存在，但是一般确定了z之后，求解就很容易了。  
### 2.1思想
EM是一种解决存在隐含变量优化问题的有效方法。既然不能直接最大化$L(\theta)$,我们可以不断建立$L(\theta)$的下界(E步),然后不断优化下界(M步)。

### 2.2分析
事实上，$z_i$也是个随机变量，对于每一个$y_i$,都有多种分类的情况。设第i个样本$y_i$在z上的概率分布为$Q_i(z_i)$,即$Q_i(z_i=j)$表示$y_i$被划分到类j的概率，因此有$\sum\limits_{z_i}Q_i(z_i)=1$。   
(如果z是连续性的，那么$Q_i(z_i)$是概率密度函数，需要将求和符号换做积分符号.)  
比如要将班上的同学聚类，假设隐变量z是身高，那么就是连续的高斯分布。如果隐变量是男女，那么就是伯努利分布了。   
进行变换得到以下公式：  
\begin{align}
\sum_{i=1}^n\log p(y_i;\theta)&=\sum_{i=1}^n\log\sum_{z_i}p(y_i,z_i;\theta)\qquad(1)\\
&=\sum_{i=1}^n\log\sum\limits_{z_i}Q_i(z_i)\dfrac{p(y_i,z_i;\theta)}{Q_i(z_i)}\quad(2)\\
&\ge \sum_{i=1}^n\sum\limits_{z_i}Q_i(z_i)\log\dfrac{p(y_i,z_i;\theta)}{Q_i(z_i)}\quad(3)
\end{align}  

这里，由(1)到(2)式比较直接，分子分母同乘以一个相等的；(2)到(3)利用了Jensen不等式，log(x)是凹函数，这里x=$\dfrac{p(y_i,z_i;\theta)}{Q_i(z_i)}$.

这个过程可以看作是对$L(\theta)$求了下界。假设$\theta$已经给定，那么$L(\theta)$的值就取决于$Q_i(z_i)$和$p(y_i,z_i;\theta)$了。我们通过调整这两个概率使下界不断上升，逼近$L(\theta)$的值。 
  
  
由Jensen不等式可知，要想等式成立，随机变量x=$\dfrac{p(y_i,z_i;\theta)}{Q_i(z_i)}$应为常数。故我们有：  
$$\dfrac{p(y_i,z_i;\theta)}{Q_i(z_i)}=c,\qquad c为常数$$

已知$\sum\limits_{z_i}Q_i(z_i)=1$，我们对上式进行变换并关于$z_i$求和得：
$$\sum\limits_{z_i}p(y_i,z_i;\theta)=c$$
代回原式解得：
\begin{align}
Q_i(z_i)&=\dfrac{p(y_i,z_i;\theta)}{\sum\limits_{z_i}p(y_i,z_i;\theta)}\\
&=\dfrac{p(y_i,z_i;\theta)}{p(y_i;\theta)}\\
&=p(z_i|y_i;\theta)
\end{align}

至此，我们推出了在固定$\theta$之后，$Q_i(z_i)$的计算公式就是后验概率，解决了$Q_i(z_i)$的选择问题，建立了$L(\theta)$的下界。接下来的M步，就是在给定$Q_i(z_i)$后，调整$\theta$,去极大化$L(\theta)$的下界。

### 2.3总结
一般的EM算法步骤如下：  
循环重复直至收敛{  
(E步)对于每一个i，计算  
$$Q_i(z_i):=p(z_i|y_i;\theta)$$
也等价于最大化$$E_z[\log\dfrac{p(y_i,z_i;\theta)}{Q_i(z_i)}]$$
(M步)计算  
$$\theta:=arg\max\limits_{\theta}\sum_{i=1}^n\sum\limits_{z_i}Q_i(z_i)\log\dfrac{p(y_i,z_i;\theta)}{Q_i(z_i)}$$
}

### 2.4 另一种推导方法
我们面对一个含有隐变量的概率模型，目标是极大化观测数据(不完全数据)Y关于参数$\theta$的对数似然函数，即极大化：  

$$L(\theta)=\log P(Y|\theta)=\log \sum_ZP(Y,Z|\theta)=\log (\sum_ZP(Z|\theta)P(Y|Z,\theta))$$  
这一极大化的主要困难是上式中包含有未观测数据并包含和的对数。  
事实上，EM算法是通过迭代逐步近似极大化$L(\theta)$的。假设在第i次迭代后$\theta$的估计值是$\theta^{(i)}$,我们希望新的估计值能使$L(\theta)$增加，即
$L(\theta)>L(\theta^{(i)})$,并逐步达到最大值，为此，考虑两者的差：  

$$L(\theta)-L(\theta^{(i)})=\log (\sum_ZP(Z|\theta)P(Y|Z,\theta))-\log P(Y|\theta^{(i)})$$

利用Jensen不等式，得到其下界：  
\begin{align}
L(\theta)-L(\theta^{(i)})&=\log (\sum_ZP(Z|\theta)P(Y|Z,\theta))-P(Y|\theta^{(i)})\\
&=\log (\sum_ZP(Z|Y,\theta^{(i)})\dfrac{P(Z|\theta)P(Y|Z,\theta))}{P(Z|Y,\theta^{(i)})}-\log P(Y|\theta^{(i)})\\
&=\log (\sum_ZP(Z|Y,\theta^{(i)})\dfrac{P(Z|\theta)P(Y|Z,\theta))}{P(Z|Y,\theta^{(i)})P(Y|\theta^{(i)})}\\
&\ge \sum_ZP(Z|Y,\theta^{(i)}) \log\dfrac{P(Z|\theta)P(Y|Z,\theta))}{P(Z|Y,\theta^{(i)})P(Y|\theta^{(i)})}
\end{align}

## 3.EM收敛性证明
假定$\theta^{(t)}$和$\theta^{(t+1)}$是EM第t次和第t+1次迭代后的结果。如果我们证明了$l(\theta^{(t)})\le l(\theta^{(t+1)})$,也就是说极大似然估计单调增加，那么我们最终会到达极大似然估计的最大值。  
证明如下：  
选定$\theta^{(t)}$后，我们得到E步
$$Q_i^{(t)}(z_i):=p(z_i|x_i;\theta^{(t)})$$
这一步保证了在给定$\theta^{(t)}$时，Jensen不等式成立，也就是
$$l(\theta^{(t)})=\sum_{i=1}^n\sum\limits_{z_i}Q_i^{(t)}(z_i)\log\dfrac{p(x_i,z_i;\theta^{(t)})}{Q_i^{(t)}(z_i)}$$
然后进行M步，固定$Q_i^{(t)}(z_i)$,并将$\theta^{(t)}$视作变量，对上面的$l(\theta^{(t)})$求导后，得到$\theta^{(t+1)}$,这样经过一些推导会有以下式子成立：  


\begin{align}
l(\theta^{(t+1)})&\ge\sum_{i=1}^n\sum\limits_{z_i}Q_i^{(t)}(z_i)\log\dfrac{p(x_i,z_i;\theta^{(t+1)})}{Q_i^{(t)}(z_i)}\\
&\ge\sum_{i=1}^n\sum\limits_{z_i}Q_i^{(t)}(z_i)\log\dfrac{p(x_i,z_i;\theta^{(t)})}{Q_i^{(t)}(z_i)}\\
&=l(\theta^{(t)})
\end{align}